In [25]:
"""Bokeh Visualization Template

This template is a general outline for turning your data into a 
visualization using Bokeh.
"""
# Data handling
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import glob, os



# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import output_file
from bokeh.plotting import figure, show

In [26]:
import pandas as pd
import numpy as np
import glob, os

# Get CSV files list from a folder
path = 'data/sales_data'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
df   = pd.concat(df_list, ignore_index=True)



#Bovenstaande code selecteert een map in path, en zet alle csv bestanden in een dataframe genaamd df, https://sparkbyexamples.com/pandas/pandas-read-multiple-csv-files/

In [27]:
df = df[["Transaction Date", "Transaction Type", "Product id", "Sku Id", "Buyer Country", "Buyer Postal Code", "Amount (Merchant Currency)"]]
#selecteert de rijen die van belang zijn (zie opdracht document)


df = df.loc[(df['Transaction Type'] == 'Charge') & (df['Product id'] == 'com.vansteinengroentjes.apps.ddfive')]
#selecteert specfieke rijen met de waarden die overeenkomen hier boven (zie opdracht document)



In [28]:
df.isna().sum()/len(df)*100

#laat het percentage NaN values zijn.


Transaction Date              0.000000
Transaction Type              0.000000
Product id                    0.000000
Sku Id                        0.000000
Buyer Country                 0.000000
Buyer Postal Code             0.323625
Amount (Merchant Currency)    0.000000
dtype: float64

In [29]:
df = df.dropna()

#dropt alle rijen waar NaN in voorkomt, https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html



In [30]:
df.isna().sum()/len(df)*100

#laat zien dat er nu geen 1 rij NaN type heeft. 


Transaction Date              0.0
Transaction Type              0.0
Product id                    0.0
Sku Id                        0.0
Buyer Country                 0.0
Buyer Postal Code             0.0
Amount (Merchant Currency)    0.0
dtype: float64

In [31]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df_sales =df
#vertaalt de transaction date naar leesbaar data voor pandas dataframe.


In [32]:
# Determine where the visualization will be rendered
# output_file('filename.html')  # Render to static HTML, or 
output_notebook()  # Render inline in a Jupyter Notebook

#selecteert de rijen transaction date en amount
date_amount_data = df_sales[["Transaction Date", "Amount (Merchant Currency)"]]
# amount_data = df_sales["Amount (Merchant Currency)"]


#maakt cds bestanden voor sales per dag
date_data_cds = ColumnDataSource(date_amount_data.groupby('Transaction Date')['Amount (Merchant Currency)'].sum().to_frame().reset_index())


# amount_data_cds = ColumnDataSource(amount_data)



# Set up the figure(s)
# Create and configure the figure
fig = figure(x_axis_type='datetime',
             plot_height=300, plot_width=600,
             title='Western Conference Top 2 Teams Wins Race, 2017-18',
             x_axis_label='Date', y_axis_label='Amount',
             toolbar_location=None)

# Connect to and draw the data
# Render the race as step lines

fig.step('Transaction Date', 'Amount (Merchant Currency)', 
         color='#CE1141', legend='Amount', 
         source=date_data_cds)

#volgende lijnen gaan over het toevoegen van een nieuwe graph
# fig.step('stDate', 'gameWon', 
#          color='#006BB6', legend='x', 
#          source=warriors_cds)

# Move the legend to the upper left corner
fig.legend.location = 'top_left'

# Organize the layout

# Preview and save 
show(fig)  # See what I made, and save if I like it


Loading BokehJS ...